In [119]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from matplotlib.widgets import Button

# Problem1

In [ ]:
def create_gaussian_kernel(k, s):
    """
    Create a 2D Gaussian kernel.
    """
    ax = np.arange(-k // 2 + 1., k // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * s**2))
    return kernel / np.sum(kernel)
def apply_convolution(image, kernel):
    """
    Apply convolution to the image using the given kernel.
    """
    m, n = kernel.shape
    y, x = image.shape
    y = y - m + 1
    x = x - n + 1
    output = np.zeros((y,x))
    for i in range(y):
        for j in range(x):
            output[i,j] = np.sum(image[i:i+m, j:j+n]*kernel)
    return output
def gradient_edge_detector(image, sigma):
    """
    Perform gradient-based edge detection as per PDF instructions.
    """
    # Create Gaussian kernel
    kernel_size = int(6 * sigma + 1)  # Ensure kernel size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    gaussian_kernel = create_gaussian_kernel(kernel_size, sigma)
    
    # Smooth the image with a Gaussian filter
    smoothed = apply_convolution(image, gaussian_kernel)
    
    # Compute horizontal and vertical derivatives
    dx_kernel = np.array([[-1, 0, 1]])
    dy_kernel = np.array([[-1], [0], [1]])
    
    grad_x = apply_convolution(smoothed, dx_kernel)
    grad_y = apply_convolution(smoothed, dy_kernel)
    
    # Ensure grad_x and grad_y have the same shape
    min_shape = (min(grad_x.shape[0], grad_y.shape[0]), min(grad_x.shape[1], grad_y.shape[1]))
    grad_x = grad_x[:min_shape[0], :min_shape[1]]
    grad_y = grad_y[:min_shape[0], :min_shape[1]]
    
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)
    
    return magnitude, orientation
def plot_results(image, magnitude, orientation):
    """
    Plot the original image, gradient magnitude, and gradient orientation.
    """
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(magnitude, cmap='gray')
    plt.title('Gradient Magnitude')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    step = 10  # Adjust this to change the density of arrows
    y, x = np.mgrid[step//2:magnitude.shape[0]:step, step//2:magnitude.shape[1]:step]
    u = np.cos(orientation[y, x])
    v = np.sin(orientation[y, x])
    plt.quiver(x, y, u, v, color='r', angles='xy', scale_units='xy', scale=0.1)
    plt.title('Gradient Orientation')
    plt.gca().invert_yaxis()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
#Load and preprocess the image
image = np.array(Image.open('./1.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Set parameter
sigma = 1.0

# Apply gradient-based edge detection
magnitude, orientation = gradient_edge_detector(image, sigma)
# Plot results
plot_results(image, magnitude, orientation)
# You can experiment with different sigma values here
# sigmas = [0.5, 1.0, 2.0]
# for sigma in sigmas:
#     magnitude, orientation = gradient_edge_detector(image, sigma)
#     plot_results(image, magnitude, orientation)
#     print(f"Results for sigma = {sigma}")

# Problem 1.2

In [ ]:
def edge_detection(image, threshold=0.1):
    """
    Perform edge detection using the gradient method and apply thresholding.
    """
    magnitude, _ = gradient_edge_detector(image, sigma=1.0)
    edge_map = (magnitude > threshold).astype(np.uint8)
    return edge_map

def trace_boundary(edge_map, start_point):
    """
    Trace the boundary of an object starting from a given point.
    """
    directions = [(-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1)]
    boundary = [start_point]
    current = start_point
    
    for _ in range(1000):  # Limit to prevent infinite loop
        for dx, dy in directions:
            next_point = (current[0] + dx, current[1] + dy)
            if 0 <= next_point[0] < edge_map.shape[0] and 0 <= next_point[1] < edge_map.shape[1]:
                if edge_map[next_point] == 1 and next_point not in boundary:
                    boundary.append(next_point)
                    current = next_point
                    break
        if current == start_point or len(boundary) > 1 and boundary[-1] == boundary[0]:
            break
    
    return boundary

def find_edge_points(edge_map, num_points=10):
    """
    Find a set of points on the edges automatically.
    """
    edge_points = np.argwhere(edge_map == 1)
    if len(edge_points) > num_points:
        return edge_points[np.linspace(0, len(edge_points) - 1, num_points, dtype=int)]
    else:
        return edge_points

def plot_results(image, edge_map, boundaries, start_points):
    """
    Plot the original image, edge map, and traced boundaries.
    """
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    ax2.imshow(edge_map, cmap='gray')
    ax2.set_title('Edge Map')
    ax2.axis('off')
    
    ax3.imshow(image, cmap='gray')
    for boundary in boundaries:
        boundary = np.array(boundary)
        ax3.plot(boundary[:, 1], boundary[:, 0], 'r-', linewidth=2)
    for point in start_points:
        ax3.plot(point[1], point[0], 'go', markersize=8)  # Green dots for start points
    ax3.set_title('Traced Boundaries')
    ax3.axis('off')
    
    plt.tight_layout()
    plt.show()

# Load and preprocess the image
image = np.array(Image.open('./1.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Perform edge detection
edge_map = edge_detection(image)

# Automatically find edge points
start_points = find_edge_points(edge_map)

# Trace boundaries from selected points
boundaries = [trace_boundary(edge_map, tuple(point)) for point in start_points]

# Plot final results
plot_results(image, edge_map, boundaries, start_points)

print("Green dots on the final image indicate the automatically selected starting points for boundary tracing.")
print("Red lines show the traced boundaries from these points.")
print("Note: This is an automatic demonstration. In a real scenario, you would manually select these points.")

In [ ]:
def edge_detection(image, threshold=0.1):
    """
    Perform edge detection using the gradient method and apply thresholding.
    """
    magnitude, _ = gradient_edge_detector(image, sigma=1.0)
    edge_map = (magnitude > threshold).astype(np.uint8)
    return edge_map

def trace_boundary(edge_map, start_point):
    """
    Trace the boundary of an object starting from a given point.
    """
    directions = [(-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1)]
    boundary = [start_point]
    current = start_point
    
    for _ in range(1000):  # Limit to prevent infinite loop
        for dx, dy in directions:
            next_point = (current[0] + dx, current[1] + dy)
            if 0 <= next_point[0] < edge_map.shape[0] and 0 <= next_point[1] < edge_map.shape[1]:
                if edge_map[next_point] == 1 and next_point not in boundary:
                    boundary.append(next_point)
                    current = next_point
                    break
        if current == start_point or len(boundary) > 1 and boundary[-1] == boundary[0]:
            break
    
    return boundary

def display_image_with_grid(image):
    """
    Display the image with a grid overlay and coordinate labels.
    """
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(image, cmap='gray')
    
    # Add grid
    ax.grid(which='major', color='r', linestyle='-', linewidth=0.5, alpha=0.5)
    
    # Set ticks and labels
    step = 25  # Adjust this value to change the grid density
    ax.set_xticks(np.arange(0, image.shape[1], step))
    ax.set_yticks(np.arange(0, image.shape[0], step))
    ax.set_xticklabels(np.arange(0, image.shape[1], step))
    ax.set_yticklabels(np.arange(0, image.shape[0], step))
    
    plt.title("Image with Coordinate Grid")
    plt.show()

def manual_coordinate_input(image_shape):
    """
    Allow manual input of coordinates.
    """
    points = []
    while True:
        try:
            x = int(input("Enter x-coordinate (or press Enter to finish): "))
            y = int(input("Enter y-coordinate: "))
            if 0 <= x < image_shape[1] and 0 <= y < image_shape[0]:
                points.append((y, x))  # Store as (row, col)
                print(f"Point ({x}, {y}) added.")
            else:
                print("Coordinates out of image bounds. Please try again.")
        except ValueError:
            break  # Exit loop if input is not a number (i.e., user pressed Enter)
    return points

def plot_results(image, edge_map, boundaries, start_points):
    """
    Plot the original image, edge map, and traced boundaries with start points.
    """
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 7))
    
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    ax2.imshow(edge_map, cmap='gray')
    ax2.set_title('Edge Map')
    ax2.axis('off')
    
    ax3.imshow(image, cmap='gray')
    for boundary in boundaries:
        boundary = np.array(boundary)
        ax3.plot(boundary[:, 1], boundary[:, 0], 'r-', linewidth=2)
    for point in start_points:
        ax3.plot(point[1], point[0], 'go', markersize=8)  # Green dots for start points
    ax3.set_title('Traced Boundaries')
    ax3.axis('off')
    
    plt.tight_layout()
    plt.show()

# Load and preprocess the image
image = np.array(Image.open('./1.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Display the image with grid
display_image_with_grid(image)

# Perform edge detection
edge_map = edge_detection(image)

# Manual coordinate input
print(f"Image shape is {image.shape}")
print("Enter the coordinates of boundary points using the grid as reference.")
print("Press Enter without inputting coordinates to finish.")
selected_points = manual_coordinate_input(image.shape)

# Trace boundaries from selected points
boundaries = [trace_boundary(edge_map, point) for point in selected_points]

# Plot final results
plot_results(image, edge_map, boundaries, selected_points)

print("Green dots on the final image indicate your manually selected starting points for boundary tracing.")
print("Red lines show the traced boundaries from these points.")